In [29]:
import pandas as pd

# Direct display of the pre-study CSV
df_csv = pd.read_csv(model_pre_study_path)
if df_csv.empty or "r2" not in df_csv.columns or df_csv["r2"].dropna().empty:
    print("CSV file is empty or missing valid 'r2' values.")
else:
    # Sort by R² descending
    df_csv = df_csv.sort_values("r2", ascending=False).reset_index(drop=True)
    
    # Conditional highlighting for Top 3 rows
    def highlight_top3(row):
        color = ""
        if row.name == 0:
            color = "background-color: #84e193;"   # Very light green for top 1
        elif row.name == 1:
            color = "background-color: #b6f5c1;"   # Light green for top 2
        elif row.name == 2:
            color = "background-color: #dafbe9;"   # Pale green for top 3
        return [color] * len(row)

    print("=== PRE-STUDY MODEL METRICS (CSV) ===")
    display(df_csv.style.apply(highlight_top3, axis=1))


=== PRE-STUDY MODEL METRICS (CSV) ===


,model,mae,rmse,r2
0,CatBoost + Optuna CV (Top RF Features) – optimized,34139.046892,48097.923126,0.943577
1,XGBoost + Optuna CV (Top RF Features) – optimized,39705.093990,56732.458564,0.921501
2,XGBoost + Optuna CV (All Features) – optimized,40994.594274,58560.847120,0.916360
3,CatBoost + Optuna RepeatedCV (Top RF Features) - RepeatedKFold,41495.974929,58790.477978,0.915702
4,LightGBM + Optuna CV (Top RF Features) – optimized,42008.018285,60202.111523,0.911606
5,CatBoost + Optuna RepeatedCV (All Features) - RepeatedKFold,44686.006207,61422.225706,0.907986
6,CatBoost + Optuna CV (All Features) – optimized,47541.829840,65274.903156,0.896081
7,LightGBM + Optuna CV (Top RF Features) – optimized,45872.499214,66342.024324,0.892656
8,CatBoost + Optuna CV (Top RF Features) – optimized,46634.307739,66384.667315,0.892518
9,LightGBM + Optuna CV (All Features) – optimized,46632.281438,68248.443936,0.886398


In [30]:
import os
import sqlite3
import pandas as pd

# Dynamically search for the path to metrics.db
current_dir = os.getcwd()
project_root = current_dir
while not os.path.exists(os.path.join(project_root, "database", "metrics.db")):
    parent = os.path.dirname(project_root)
    if parent == project_root:
        raise FileNotFoundError("Unable to find database/metrics.db in any parent folders.")
    project_root = parent

db_path = os.path.join(project_root, "database", "metrics.db")

# Read the database
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM model_evaluations", conn)
conn.close()

# Add the is_test column
df["is_test"] = df["model"].str.contains("TEST|test", case=False, na=False)
df = df.sort_values(by=["is_test", "r2"], ascending=[True, False]).reset_index(drop=True)

# Reorder columns to put dataset before model
col_order = ['id', 'timestamp', 'dataset', 'model', 'experiment', 'cleaning_version_id',
             'mae', 'rmse', 'r2', 'is_test']
df = df[[col for col in col_order if col in df.columns]]

# Find the indices for the top 3 models (excluding test models)
top = df[df["is_test"] == False].sort_values("r2", ascending=False).head(3)
top1_idx = top.index[0]
top2_idx = top.index[1] if len(top) > 1 else None
top3_idx = top.index[2] if len(top) > 2 else None

# Row coloring function for Top 3
def highlight_top3(row):
    if row.name == top1_idx:
        color = 'background-color: #b9fcb9'      # Dark green
    elif row.name == top2_idx:
        color = 'background-color: #d6f9d6'      # Medium green
    elif row.name == top3_idx:
        color = 'background-color: #f2fff2'      # Light green
    else:
        color = ''
    return [color] * len(row)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)

from IPython.display import display
print("=== FULL TABLE (TOP 1, 2, 3 IN GREENS, DATASET BEFORE MODEL) ===")
display(df.style.apply(highlight_top3, axis=1))


=== FULL TABLE (TOP 1, 2, 3 IN GREENS, DATASET BEFORE MODEL) ===


,id,timestamp,dataset,model,experiment,cleaning_version_id,mae,rmse,r2,is_test
0,41,2025-06-29 18:30:56,immoweb_real_estate_cleaned_for_ml_20250629_1826.csv,CatBoost + Optuna CV (All Features),CatBoost with Optuna (All Features),202506291826,20867.176266,28876.028355,0.980897,False
1,52,2025-06-29 18:39:31,immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,CatBoost + Optuna CV (All Features),CatBoost with Optuna (All Features),202506291834,21701.841801,30148.209880,0.979177,False
2,22,2025-06-29 17:49:29,immoweb_real_estate_cleaned_for_ml_20250629_1742.csv,CatBoost + Optuna CV (Top RF Features),CatBoost with Optuna (Top RF Features),202506291742,28785.609199,40088.000130,0.963182,False
3,21,2025-06-29 17:49:29,immoweb_real_estate_cleaned_for_ml_20250629_1742.csv,CatBoost + Optuna CV (All Features),CatBoost with Optuna (All Features),202506291742,35438.389057,49381.156058,0.944133,False
4,10,2025-06-29 16:44:52,immoweb_real_estate_cleaned_for_ml_20250629_1641.csv,CatBoost + Optuna CV (All Features),CatBoost with Optuna (All Features),202506291641,45214.335325,62794.750156,0.909661,False
5,42,2025-06-29 18:30:56,immoweb_real_estate_cleaned_for_ml_20250629_1826.csv,CatBoost + Optuna CV (Top RF Features),CatBoost with Optuna (Top RF Features),202506291826,44879.789454,63089.664714,0.908810,False
6,53,2025-06-29 18:39:31,immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,CatBoost + Optuna CV (Top RF Features),CatBoost with Optuna (Top RF Features),202506291834,46390.435268,65021.643925,0.903140,False
7,11,2025-06-29 16:44:52,immoweb_real_estate_cleaned_for_ml_20250629_1641.csv,CatBoost + Optuna CV (Top RF Features),CatBoost with Optuna (Top RF Features),202506291641,46839.046978,65280.540746,0.902367,False
8,2,2025-06-29 16:42:13,immoweb_real_estate_cleaned_for_ml_20250629_1641.csv,Random Forest (All Features),Random Forest (All Features),202506291641,68324.023924,101428.245931,0.764228,False
9,13,2025-06-29 17:43:58,immoweb_real_estate_cleaned_for_ml_20250629_1742.csv,Random Forest (All Features),Random Forest (All Features),202506291742,68324.023924,101428.245931,0.764228,False
